In [1]:
# -----------------------------------------------------------
#  VWAP Breakout Strategy • intraday • backtesting.py
# -----------------------------------------------------------
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import time, datetime, timedelta
from backtesting import Backtest, Strategy

def add_vwap(
    df: pd.DataFrame,
    time_col: str | None = None,
    price_col: str = "Close",
    vol_col: str = "Volume",
    **parse_kwargs                # ← forwarded to pd.to_datetime
) -> pd.DataFrame:
    df = df.copy()

    # 1 ─ ensure DatetimeIndex
    if time_col is not None:
        df[time_col] = pd.to_datetime(df[time_col], errors="coerce", **parse_kwargs)
        df = df.set_index(time_col)

    if not isinstance(df.index, pd.DatetimeIndex) or df.index.hasnans:
        raise TypeError("Index (or `time_col`) must be datetime-like and parse without NaT")

    # 2 ─ price to weight by volume
    tp = ((df["High"] + df["Low"] + df["Close"]) / 3) if price_col.lower() == "typical" else df[price_col]

    # 3 ─ VWAP per calendar day
    day = df.index.normalize()
    cum_vol = df[vol_col].groupby(day).cumsum()
    cum_pv  = (tp * df[vol_col]).groupby(day).cumsum()
    df["VWAP"] = cum_pv / cum_vol
    return df

def fetch_indian_stock_data(symbol: str, period: str = "1y", interval: str = "15m") -> pd.DataFrame:
    """
    Fetch Indian stock data using yfinance
    
    Parameters:
    -----------
    symbol : str
        Stock symbol (e.g., 'RELIANCE.NS' for NSE, 'RELIANCE.BO' for BSE)
    period : str
        Data period ('1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max')
    interval : str
        Data interval ('1m', '2m', '5m', '15m', '30m', '60m', '90m', '1h', '1d', '5d', '1wk', '1mo', '3mo')
    
    Returns:
    --------
    pd.DataFrame
        OHLCV data with datetime index
    """
    print(f"Fetching {symbol} data...")
    ticker = yf.Ticker(symbol)
    data = ticker.history(period=period, interval=interval)
    
    # Reset index to have datetime as column
    data = data.reset_index()
    data.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
    
    # Remove any rows with zero volume (market closed periods)
    data = data[data['Volume'] > 0].copy()
    
    print(f"✓ Fetched {len(data)} rows of data")
    print(f"✓ Date range: {data['Datetime'].min()} to {data['Datetime'].max()}")
    
    return data

/Users/mohitmandawat/Coding/CodeTrading/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
def add_weekly_vwap(
    df: pd.DataFrame,
    *,
    time_col: str | None = None,
    price_col: str = "Close",
    vol_col:   str = "Volume",
    freq:      str = "D",           # "D" (daily) or "W-MON", "W-FRI", …
    **parse_kwargs,                 # forwarded to pd.to_datetime
) -> pd.DataFrame:
    """
    Append a VWAP column that resets every *freq* period.

    Parameters
    ----------
    df : pd.DataFrame
        Must contain `price_col`, `vol_col`, and (optionally) High/Low if
        price_col="Typical".
    time_col : str | None
        Pass the name of the timestamp column if the index is not already
        Datetime-like.
    price_col : {"Close", "Typical", ...}
        Price to weight by volume.  "Typical" uses (High+Low+Close)/3.
    vol_col : str
        Volume column name.
    freq : str, default "D"
        Pandas offset alias that defines VWAP reset boundary.
        Examples: "D" (daily), "W-MON" (weekly ending Monday), "M" (monthly).
    **parse_kwargs
        Extra arguments passed to `pd.to_datetime` when parsing `time_col`.

    Returns
    -------
    pd.DataFrame
        Copy of `df` with an added 'VWAP' column.
    """
    df = df.copy()

    # -- 1. ensure DatetimeIndex --------------------------------------------
    if time_col is not None:
        df[time_col] = pd.to_datetime(df[time_col], errors="coerce", **parse_kwargs)
        df = df.set_index(time_col)

    if not isinstance(df.index, pd.DatetimeIndex) or df.index.hasnans:
        raise TypeError("Index (or `time_col`) must be datetime-like and parse without NaT")

    # -- 2. choose the price series -----------------------------------------
    if price_col.lower() == "typical":
        tp = (df["High"] + df["Low"] + df["Close"]) / 3
    else:
        tp = df[price_col]

    # -- 3. labels defining each VWAP bucket --------------------------------
    # floor to the start of each period (e.g. Monday 00:00 if freq="W-MON")
    labels = df.index.to_period(freq).to_timestamp()

    # cumulative sums within each period
    cum_vol = df[vol_col].groupby(labels).cumsum()
    cum_pv  = (tp * df[vol_col]).groupby(labels).cumsum()

    df["VWAP"] = cum_pv / cum_vol
    return df


In [3]:

# -------- 3. strategy --------------------------------------
class VWAPBreakout(Strategy):
    """
    • Long  when Close crosses above VWAP
    • Short when Close crosses below VWAP
    Optional filter: take longs only if Close > day’s Open (bull bias)
    Always flat by 15:45‒16:00 ET bar
    """

    # --- configurable parameters (can be optimised) --------
    intraday_close_time = time(15, 45)  # last bar before close (interval = 15m)
    atr_stop = 1.5  # e.g. 1.5 to use ATR trailing stop, None = no stop

    def init(self):
        # pre-compute ATR if a trailing stop is requested
        if self.atr_stop:
            self.atr = self.I(self._atr, self.data.High, self.data.Low, self.data.Close, 14)

    # ---- utility: ATR (simple True Range MA)
    @staticmethod
    def _atr(h, l, c, n):
        tr = np.maximum.reduce([h[1:] - l[1:], abs(h[1:] - c[:-1]), abs(l[1:] - c[:-1])])
        atr = pd.Series(tr).rolling(n).mean()
        return np.append([np.nan], atr)  # align length

    # -------------------------------------------------------
    def next(self):
        #i = len(self.data.Close) - 1  # current bar index

        close = self.data.Close[-1]
        vwap  = self.data.VWAP[-1]

        # --- determine daily open price for filters ----------
        current_day = self.data.index[-1].date()
        day_open = self.data.Open[self.data.index.date == current_day][0]

        # -------- entry logic --------------------------------
        if not self.position:

            # LONG
            if (close > vwap) and (close > day_open): #and close>self.data.Open[-1]
                self.buy()

            # SHORT
            elif (close < vwap) and (close < day_open): # and close<self.data.Open[-1]
                self.sell()

        if self.position and self.atr_stop:          # <-- note lower-case .position
            price = self.data.Close[-1]     
            atr = self.atr[-1]
            trail = self.atr_stop * atr

            # tighten stop on every active trade
            for trade in self.trades:
                if trade.is_long:
                    new_sl = price - trail
                    if trade.sl is None or new_sl > trade.sl:
                        trade.sl = new_sl
                else:                      # short trade
                    new_sl = price + trail
                    if trade.sl is None or new_sl < trade.sl:
                        trade.sl = new_sl

        # -------- intraday exit: flatten before close --------
        if self.position:
            bar_time = self.data.index[-1].time()
            if bar_time >= self.intraday_close_time:
                self.position.close()

In [ ]:
# ============================================================================
# FETCH RELIANCE DATA FROM NSE USING YFINANCE
# ============================================================================

# Fetch Reliance Industries data from NSE
df = fetch_indian_stock_data("RELIANCE.NS", period="1y", interval="15m")

print(f"\nData shape: {df.shape}")
print(f"Sample data:")
print(df.head())

# Add VWAP to the data
df = add_vwap(df, time_col="Datetime")  # yfinance uses 'Datetime' column

print(f"\nData with VWAP:")
print(df.head())
print(f"\nVWAP statistics:")
print(f"  Min VWAP: ₹{df['VWAP'].min():.2f}")
print(f"  Max VWAP: ₹{df['VWAP'].max():.2f}")
print(f"  Mean VWAP: ₹{df['VWAP'].mean():.2f}")

# Use a subset for backtesting (adjust range as needed)
data_subset = df.iloc[500:2000] if len(df) > 2000 else df.iloc[100:]
print(f"\nUsing data subset: {len(data_subset)} rows for backtesting")

# ============================================================================
# ALTERNATIVE: CSV DATA (COMMENTED OUT)
# ============================================================================
# Uncomment below lines to use CSV data instead of yfinance
#df = pd.read_csv("TSLA.USUSD_Candlestick_15_M_BID_01.06.2024-28.06.2025.csv")
#df = pd.read_csv("BTCUSD_Candlestick_15_M_BID_01.06.2024-28.06.2025.csv")
#df = pd.read_csv("AAPL.USUSD_Candlestick_1_D_BID_01.06.2020-28.06.2025.csv")
#df = pd.read_csv("TSLA.USUSD_Candlestick_15_M_BID_30.06.2022-28.06.2025.csv")
#df = add_vwap(df, time_col="Gmt time", dayfirst=True, format="%d.%m.%Y %H:%M:%S.%f")

# ============================================================================
# BACKTESTING SETUP FOR INDIAN MARKET
# ============================================================================

bt = Backtest(
    data_subset,
    VWAPBreakout,
    cash=100_000,
    commission=0.001,  # 0.1% commission typical for Indian brokers
    exclusive_orders=True,  # only one position at a time
    trade_on_close=True,    # act on bar close prices
)

import matplotlib.pyplot as plt

print("\n" + "="*60)
print("BACKTESTING RELIANCE INDUSTRIES (NSE) - VWAP BREAKOUT STRATEGY")
print("="*60)

stats = bt.run()
print(stats)
bt.plot(show_legend=False)

In [ ]:
# ============================================================================
# PARAMETER OPTIMIZATION FOR RELIANCE (INDIAN MARKET)
# ============================================================================

print("Starting parameter optimization for Reliance Industries...")
print("Testing ATR stop multipliers from 1.0 to 3.0...")
print("This may take a few minutes...\n")

stats_best, heatmap = bt.optimize(
    atr_stop = [round(x, 2) for x in np.arange(1.0, 3.01, 0.25)],
    maximize = "Sharpe Ratio",
    return_heatmap = True,      # keep all runs, not just the best
)

# Extract best results
best_atr = stats_best._strategy.atr_stop
best_ret = stats_best["Return [%]"]
best_sharpe = stats_best["Sharpe Ratio"]
best_dd = stats_best["Max. Drawdown [%]"]
best_win_rate = stats_best["Win Rate [%]"]
best_trades = stats_best["# Trades"]

print("\n" + "="*60)
print("OPTIMIZATION RESULTS - RELIANCE INDUSTRIES (NSE)")
print("="*60)
print(f"🎯 Best ATR Stop Multiplier : {best_atr:.2f}x")
print(f"📈 Total Return            : {best_ret:.2f}%")
print(f"📊 Sharpe Ratio            : {best_sharpe:.3f}")
print(f"📉 Maximum Drawdown        : {best_dd:.2f}%")
print(f"🎲 Win Rate                : {best_win_rate:.1f}%")
print(f"🔄 Total Trades            : {best_trades}")

# Calculate additional metrics
if best_dd != 0:
    calmar_ratio = best_ret / abs(best_dd)
    print(f"📏 Calmar Ratio            : {calmar_ratio:.3f}")

print(f"💰 Final Portfolio Value   : ₹{100000 * (1 + best_ret/100):,.0f}")
print("="*60)

In [ ]:
# ============================================================================
# DETAILED OPTIMIZATION RESULTS TABLE
# ============================================================================

print("Calculating detailed results for each ATR multiplier...")

# Process heatmap results
summary = (
    heatmap
    .rename("Sharpe Ratio")
    .reset_index()
    .rename(columns={"index": "atr_stop"})
    .sort_values("atr_stop")
)

# Calculate additional metrics for each parameter
returns = []
max_drawdowns = []
win_rates = []
total_trades = []
volatilities = []

for val in summary["atr_stop"]:
    res = bt.run(atr_stop=val)
    returns.append(res["Return [%]"])
    max_drawdowns.append(res["Max. Drawdown [%]"])
    win_rates.append(res["Win Rate [%]"])
    total_trades.append(res["# Trades"])
    volatilities.append(res["Volatility [%]"])

# Add columns to summary
summary["Return [%]"] = returns
summary["Max DD [%]"] = max_drawdowns
summary["Win Rate [%]"] = win_rates
summary["Trades"] = total_trades
summary["Volatility [%]"] = volatilities

# Calculate Calmar Ratio
summary["Calmar Ratio"] = summary["Return [%]"] / abs(summary["Max DD [%]"])

print(f"\n" + "="*80)
print("COMPLETE PARAMETER SWEEP - RELIANCE INDUSTRIES")
print("="*80)

# Display formatted table
pd.set_option('display.float_format', '{:.2f}'.format)
print(summary[['atr_stop', 'Return [%]', 'Sharpe Ratio', 'Max DD [%]', 
               'Win Rate [%]', 'Calmar Ratio', 'Trades']].to_string(index=False))

print(f"\n" + "="*40)
print("BEST PERFORMERS BY METRIC:")
print("="*40)

best_sharpe_atr = summary.loc[summary['Sharpe Ratio'].idxmax(), 'atr_stop']
best_return_atr = summary.loc[summary['Return [%]'].idxmax(), 'atr_stop']
best_calmar_atr = summary.loc[summary['Calmar Ratio'].idxmax(), 'atr_stop']
best_winrate_atr = summary.loc[summary['Win Rate [%]'].idxmax(), 'atr_stop']
lowest_dd_atr = summary.loc[summary['Max DD [%]'].idxmin(), 'atr_stop']

print(f"🏆 Best Sharpe Ratio  : ATR = {best_sharpe_atr} ({summary.loc[summary['atr_stop']==best_sharpe_atr, 'Sharpe Ratio'].values[0]:.3f})")
print(f"📈 Best Return        : ATR = {best_return_atr} ({summary.loc[summary['atr_stop']==best_return_atr, 'Return [%]'].values[0]:.1f}%)")
print(f"📏 Best Calmar Ratio  : ATR = {best_calmar_atr} ({summary.loc[summary['atr_stop']==best_calmar_atr, 'Calmar Ratio'].values[0]:.3f})")
print(f"🎲 Best Win Rate      : ATR = {best_winrate_atr} ({summary.loc[summary['atr_stop']==best_winrate_atr, 'Win Rate [%]'].values[0]:.1f}%)")
print(f"🛡️ Lowest Drawdown   : ATR = {lowest_dd_atr} ({summary.loc[summary['atr_stop']==lowest_dd_atr, 'Max DD [%]'].values[0]:.1f}%)")
print("="*40)

In [ ]:
# ============================================================================
# FINAL PERFORMANCE ANALYSIS WITH OPTIMIZED PARAMETERS
# ============================================================================

print("Generating final performance analysis with optimized parameters...")
print(f"Using ATR Stop = {best_atr}")

# Run backtest with optimal parameters
final_stats = bt.run(atr_stop=best_atr)

# Generate plot and save
bt.plot(filename='reliance_vwap_strategy_optimized.html')

print(f"\n" + "="*70)
print("📊 FINAL PERFORMANCE REPORT - RELIANCE INDUSTRIES")
print("="*70)
print(f"🏢 Company           : Reliance Industries Limited (NSE)")
print(f"📈 Strategy          : VWAP Breakout with ATR Trailing Stop")
print(f"⏱️ Timeframe         : 15-minute intraday data")
print(f"🎯 Optimal ATR Stop  : {best_atr}x")
print(f"💼 Initial Capital   : ₹1,00,000")
print(f"💰 Final Value       : ₹{100000 * (1 + final_stats['Return [%]']/100):,.0f}")

print(f"\n📈 PERFORMANCE METRICS:")
print(f"   Total Return      : {final_stats['Return [%]']:>8.2f}%")
print(f"   Sharpe Ratio      : {final_stats['Sharpe Ratio']:>8.3f}")
print(f"   Maximum Drawdown  : {final_stats['Max. Drawdown [%]']:>8.2f}%")
print(f"   Volatility        : {final_stats['Volatility [%]']:>8.2f}%")

# Calculate additional metrics
returns_annual = final_stats['Return [%]']
dd_max = abs(final_stats['Max. Drawdown [%]'])
calmar_final = returns_annual / dd_max if dd_max > 0 else float('inf')

print(f"   Calmar Ratio      : {calmar_final:>8.3f}")

print(f"\n🎲 TRADING METRICS:")
print(f"   Total Trades      : {final_stats['# Trades']:>8}")
print(f"   Win Rate          : {final_stats['Win Rate [%]']:>8.1f}%")
print(f"   Best Trade        : {final_stats['Best Trade [%]']:>8.2f}%")
print(f"   Worst Trade       : {final_stats['Worst Trade [%]']:>8.2f}%")
print(f"   Avg Trade Duration: {final_stats['Avg. Trade Duration']:>8}")

print(f"\n💡 RISK ASSESSMENT:")
if calmar_final > 1.0:
    print("   ✅ Good risk-adjusted returns (Calmar > 1.0)")
else:
    print("   ⚠️ Moderate risk-adjusted returns (Calmar < 1.0)")

if final_stats['Win Rate [%]'] > 50:
    print("   ✅ Positive win rate")
else:
    print("   ⚠️ Win rate below 50%")

if dd_max < 20:
    print("   ✅ Acceptable maximum drawdown")
else:
    print("   ⚠️ High maximum drawdown - consider risk management")

print(f"\n📝 NOTES:")
print("   • Results based on historical data - past performance doesn't guarantee future results")
print("   • Strategy includes 0.1% commission typical for Indian brokers")
print("   • Consider paper trading before live implementation")
print("   • Monitor performance regularly and adjust parameters as needed")
print("="*70)

In [ ]:
# ============================================================================
# BONUS: TRY OTHER INDIAN STOCKS
# ============================================================================

print("💡 Want to test other Indian stocks? Try these examples:")
print("="*60)

# List of popular Indian stocks
indian_stocks = {
    "RELIANCE.NS": "Reliance Industries (Current)",
    "TCS.NS": "Tata Consultancy Services",
    "HDFCBANK.NS": "HDFC Bank",
    "INFY.NS": "Infosys",
    "HINDUNILVR.NS": "Hindustan Unilever",
    "ITC.NS": "ITC Limited",
    "LT.NS": "Larsen & Toubro",
    "KOTAKBANK.NS": "Kotak Mahindra Bank",
    "BAJFINANCE.NS": "Bajaj Finance",
    "MARUTI.NS": "Maruti Suzuki"
}

print("Popular NSE stocks to test:")
for symbol, name in indian_stocks.items():
    print(f"  {symbol:<15} - {name}")

print(f"\n📝 To test a different stock, replace 'RELIANCE.NS' in the data fetching cell with any symbol above.")
print(f"📝 You can also modify the period and interval:")
print(f"   • Periods: '1mo', '3mo', '6mo', '1y', '2y', '5y'")
print(f"   • Intervals: '5m', '15m', '30m', '1h', '1d'")

print(f"\n🔄 Example: To test TCS with 5-minute data:")
print(f"   df = fetch_indian_stock_data('TCS.NS', period='6mo', interval='5m')")

print(f"\n⚠️ Note: Smaller intervals (1m, 2m, 5m) may have data limitations from yfinance.")
print("="*60)